In [1]:
import numpy as np
import os
import glob
import math
import nibabel as nib
import cv2

from pycimg import CImg
from skimage import measure
from skimage import morphology
from scipy import ndimage
from scipy.spatial import ConvexHull


In [2]:
#img_numbers = ['01', '07', '10', '11', '12', '13', '14']
img_numbers = ['07']
images = []

In [3]:
for img_num in img_numbers:
    fname = './Images/IMG_00' + img_num + '.nii.gz'
    
    foo = nib.load(fname).get_fdata()
    images.append(foo)
    print(foo.shape)

(512, 512, 145)


In [4]:
from scipy import ndimage as ndi
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
import numpy as np
from scipy import ndimage
from skimage.measure import label, regionprops

In [5]:
body_masks = []

for foo in images:
    denoised = ndi.median_filter(foo, size=3)
    
    li_thresholded = denoised > filters.threshold_li(denoised) * 0.3
    
    filled = ndi.binary_fill_holes(li_thresholded)

    width = 45
    
    processed = morphology.remove_small_holes(
        filled, 
        area_threshold=width ** 3
    )
    
    processed = morphology.remove_small_objects(
        processed, 
        min_size=width ** 3
    )
    
    labeled_image, num_features = ndimage.label(processed)

    segment_sizes = [r.area for r in regionprops(labeled_image)]
    
    sorted_segment_sizes = sorted(enumerate(segment_sizes, 1), key=lambda x: x[1], reverse=True)
    
    mask = np.zeros_like(labeled_image, dtype=bool)
    
    largest_label, _ = sorted_segment_sizes[0]
    mask[labeled_image == largest_label] = True
    
    lung_data = np.where(mask, labeled_image, 0)

    lung_reverse = 1 - lung_data

    labeled_image, num_features = ndimage.label(lung_reverse)
    
    segment_sizes = np.bincount(labeled_image.flat)
    
    largest_segment_label = np.argmax(segment_sizes[1:]) + 1
    
    largest_segment_mask = (labeled_image == largest_segment_label)
    
    largest_segment_image = largest_segment_mask.astype(int)

    body_mask = 1 - largest_segment_image

    body_masks.append(body_mask)

In [6]:
# Lab 2 - Right and left lung segmentation separately

left_markers = []
right_markers = []

for foo, body_mask in zip(images, body_masks):
    binary_img = (foo > -320).astype(int)

    binary_img = 1-binary_img
    air_regions = np.where(body_mask == 0, 0, binary_img)

    air_regions_filtered = morphology.remove_small_holes(
        air_regions, 
        area_threshold=135
    )
    
    air_regions_filtered = morphology.remove_small_objects(
        air_regions_filtered, 
        min_size=135
    )

    closed_mask = morphology.binary_closing(air_regions, morphology.ball(5))
    
    label_image, num_features = morphology.label(closed_mask, connectivity=2, return_num=True)
    
    min_lung_size = 1000  # Minimum size for lung regions
    filtered_mask = morphology.remove_small_objects(label_image, min_size=min_lung_size)
    filtered_mask[filtered_mask != 0] = 1
    
    opened_lungs = morphology.opening(filtered_mask, morphology.ball(7))

    edges = segmentation.find_boundaries(opened_lungs, mode='inner')
    
    markers = morphology.label(edges)
    
    label_image, num_labels = morphology.label(opened_lungs, connectivity=2, return_num=True)
    
    lung_regions = regionprops(label_image)
    
    left_lungs = []
    right_lungs = []
    
    for region in lung_regions:
        if region.centroid[1] < 280:
            left_lungs.append(region.label)
        else:
            right_lungs.append(region.label)
    
    left_marker = np.isin(label_image, left_lungs).astype(int)
    right_marker = np.isin(label_image, right_lungs).astype(int)

    left_markers.append(left_marker)
    right_markers.append(right_marker)

    print("Segmentation finished")

C:\Users\Admin\AppData\Local\Temp\ipykernel_11396\4289163537.py:12: UserWarning: Any labeled images will be returned as a boolean array. Did you mean to use a boolean array?
  air_regions_filtered = morphology.remove_small_holes(


Segmentation finished


In [10]:
CImg(left_marker).display();
CImg(right_marker).display();

In [7]:
left_targets = []
right_targets = []
body_targets = []

for img_number in img_numbers:
    fname = './ReferenceSegmentations/ReferenceSegmentations/LUNGS_IMG_00' + img_number +  '.nii.gz'
    
    target = nib.load(fname).get_fdata()
    print(target.shape)

    right_target = np.zeros_like(target)
    left_target = np.zeros_like(target)
    
    right_target[target == 3] = 1
    left_target[target == 2] = 1
    right_targets.append(right_target)
    left_targets.append(left_target)
                
    fname = './BodyMasks/BodyMasks/BODYMASK_IMG_00' + img_number + '.nii.gz'
    body_target = nib.load(fname).get_fdata()
    body_target = body_target.astype(np.uint8)

    body_targets.append(body_target)
    
    print("Segmentation finished")

(512, 512, 145)
Segmentation finished


In [8]:
import surface_distance
from surface_distance import metrics

for num, body_mask, body_target, left_mask, left_target, right_mask, right_target in zip(img_numbers, body_masks, body_targets, left_markers, left_targets, right_markers, right_targets): 
    right_target = right_target.astype(np.uint8)
    left_target = left_target.astype(np.uint8)
    left_marker = left_marker.astype(np.uint8)
    right_marker = right_marker.astype(np.uint8)
    body_mask = body_mask.astype(np.uint8)
    body_target = body_target.astype(np.uint8)

    coef_body = metrics.compute_dice_coefficient(body_target, body_mask)
    coef_left = metrics.compute_dice_coefficient(left_target, left_mask)
    coef_right = metrics.compute_dice_coefficient(right_target, right_mask)

    print("For image number ", num)
    print("Dice coefficient for the right lung: ", coef_right)
    print("Dice coefficient for the left lung: ", coef_left)
    print("Dice coefficient for body mask: ", coef_body)

For image number  07
Dice coefficient for the right lung:  0.0
Dice coefficient for the left lung:  0.6476069042247554
Dice coefficient for body mask:  0.9991545447929256


AttributeError: type object 'CImg' has no attribute 'display'